In [17]:
import torch
import cv2
import numpy as np
import threading

# Define your ArcFace model class
class YourArcFaceModel(torch.nn.Module):
    def __init__(self, num_classes=512):
        super(YourArcFaceModel, self).__init__()
        # Example layers for face recognition (ArcFace)
        self.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = torch.nn.Linear(28 * 28 * 64, num_classes)

    def forward(self, x):
        # Forward pass for face recognition
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(-1, 28 * 28 * 64)
        x = torch.sigmoid(self.fc1(x))
        return x

# Load the state dictionary (OrderedDict) from the model file
state_dict = torch.load(r"C:\Users\sukri\Downloads\backbone_r100.pth", map_location=torch.device('cpu'))

# Create an instance of your ArcFace model
model = YourArcFaceModel()

# Load state_dict into your model with strict=False
model.load_state_dict(state_dict, strict=False)

# Set model to evaluation mode
model.eval()

# Define face detector
face_cascade = cv2.CascadeClassifier(r"C:\Users\sukri\Downloads\archive(2)\haarcascade_frontalface_default.xml")

# Function to extract face embeddings using ArcFace
def extract_face_embedding(face):
    # Preprocess face image
    resized_face = cv2.resize(face, (112, 112))  # Example size for ArcFace
    normalized_face = resized_face.astype('float32') / 255.0  # Normalize to [0, 1]
    input_tensor = torch.tensor(normalized_face.transpose(2, 0, 1)).unsqueeze(0)  # Assuming channel-first

    # Forward pass through ArcFace model to get embeddings
    with torch.no_grad():
        embedding = model(input_tensor)

    return embedding.numpy().flatten()

# Function to calculate accuracy
def calculate_accuracy(predictions, labels):
    correct = (predictions == labels).sum()
    total = len(labels)
    accuracy = correct / total if total > 0 else 0.0
    return accuracy

# Function to process live video stream
def process_live_video():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open video capture.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            embedding = extract_face_embedding(face)
            
            # Example: Calculate accuracy based on embedding similarity
            # Here, you can replace this logic with your actual accuracy calculation
            accuracy = np.random.uniform(0.7, 0.95)  # Example accuracy, replace with actual logic
            
            # Display bounding box and accuracy
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, f"Accuracy: {accuracy:.2f}", (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        cv2.imshow('Live Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Start video processing
process_live_video()
